**Купцов Даниил Романович**

**ДЗ №1. "Как я использовал бутстрэп?"**

---

В домашнем задании мы разберём применение "наивного" бутстрэпа и попробуем построить 95%-ый доверительный интервал для оценки корреляции.
В качестве исследуемого источника данных возьмём данные макроэкономической статистики РФ из официального источника (Федеральная служба государственной статистики), а именно - данные по средней номинальной заработной плате и по индексу реальной зарплаты.

**Методология:**
* **Среднегодовая номинальная заработная плата (WAG_C_Y)** - исчисляется исходя из фонда заработной платы работников, деленного на среднесписочную численность работников. В фонд заработной платы включаются начисленные суммы в денежной и натуральной формах за отработанное время и выполненную работу, неотработанное, но оплаченное время (например, ежегодные отпуска), стимулирующие доплаты и надбавки, премии и единовременные поощрения, компенсационные выплаты, связанные с режимом работы и условиями труда.
* **Индекс реальной зарплаты (WAG_R_Y)** - реальная заработная плата характеризует объем товаров и услуг, которые можно приобрести на заработную плату в текущем периоде, исходя из цен базисного периода. Индекс реальной заработной платы исчисляется путем деления индекса номинальной заработной платы на индекс потребительских цен за один и тот же временной период.
---

1) Установим необходимые пакеты для предстоящей работы:


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns

Подключаем пакет **arch**, предназначенный для работы с моделями временных рядов с условной гетероскедастичностью и стохастической волатильностью. Из пакета мы будем использовать функции для бутстрэпа (в arch есть много хорошо реализованных разновидностей бутстрэпа). *Бутстрэп - это практический компьютерный метод исследования распределения статистик вероятностных распределений, основанный на многократной генерации выборок методом Монте-Карло на базе имеющейся выборки. Позволяет просто и быстро оценивать самые разные статистики для сложных моделей.*

In [1]:
!pip install arch
from arch.bootstrap import IIDBootstrap, IndependentSamplesBootstrap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 13.4 MB/s eta 0:00:00


2) Подгрузим источник данных "Salary":

In [11]:
data = pd.read_csv('Salary_Y.csv', sep=';')
data.head()

,YEAR,WAG_C_Y,WAG_R_Y
0,1993,58.7,100.00
1,1994,220.4,92.00
2,1995,472.4,66.24
3,1996,790.2,70.21
4,1997,950.2,73.51


3) Посчитаем точечную корреляцию:

In [20]:
wag_c_y = data['WAG_C_Y']
wag_r_y = data['WAG_R_Y']
def corr(wag_c_y,wag_r_y): #Определяем функцию.
  corr_mat = np.corrcoef(wag_c_y,wag_r_y) #Считаем корреляционную матрицу.
  return corr_mat[0, 1] #Возвращаем пользователю значение с координатами (0, 1).

corr(wag_c_y,wag_r_y) #Смотрим на получившуюся с помощью функции корреляцию между показателями.

0.9359265342548944

Теперь мы имеем одну точечную оценку для корреляции, но нам нужно применить "наивный" бутстрэп. Поэтому мы "размножим" имеющуюся одну точечную оценку на 10 тыс. "наивных" оценок, отсёчём самые маленькие/большие и получим доверительный интервал для неизвестной корреляции.

4) Построим одну бутстрэп-выборку. Нам важно взять элементы вектора  wag_c_y  и элементы вектора  wag_r_y  с одинаковыми номерами. Поэтому для того, чтобы сгенерировать случайную выборку  wag_c_y  и случайную выборку  wag_r_y  с теми же номерами, сгенерируем случайные номера:

In [22]:
rng = np.random.default_rng(111111)
obs_id = rng.choice(range(len(wag_c_y)), size=len(wag_c_y)) #Сгенерируем номера наблюдений, которые будем использовать.
obs_id

array([22, 10, 16, 19, 12, 26,  6, 16,  0, 12, 20,  8,  6, 25, 18, 20, 14,
        1, 25, 29, 14, 26,  8, 28, 28,  7, 18,  4, 22,  0])

Для того, чтобы между показателями не была потеряна связь при генерации новой выборки, "привяжем" новые выборки wag_c_y1 и wag_r_y1 к заранее сгенерированным номерам:

In [23]:
wag_c_y1 = wag_c_y[obs_id] #Возьмём случайные наблюдения в векторе wag_c_y под заранее сгенерированными местами.
wag_c_y1

22    33981.0
10     5499.0
16    18638.0
19    26822.0
12     8550.0
26    47468.0
6      1523.0
16    18638.0
0        58.7
12     8550.0
20    29960.0
8      3240.0
6      1523.0
25    43724.0
18    23693.0
20    29960.0
14    13593.0
1       220.4
25    43724.0
29    65338.0
14    13593.0
26    47468.0
8      3240.0
28    56545.0
28    56545.0
7      2223.0
18    23693.0
4       950.2
22    33981.0
0        58.7
Name: WAG_C_Y, dtype: float64

In [24]:
wag_r_y1 = wag_r_y[obs_id] #Возьмём случайные наблюдения в векторе wag_r_y под заранее сгенерированными местами.
wag_r_y1

22    186.74
10     92.76
16    165.05
19    193.49
12    115.52
26    220.25
6      49.66
16    165.05
0     100.00
12    115.52
20    202.78
8      71.98
6      49.66
25    210.16
18    178.50
20    202.78
14    153.40
1      92.00
25    210.16
29    235.95
14    153.40
26    220.25
8      71.98
28    235.25
28    235.25
7      60.04
18    178.50
4      73.51
22    186.74
0     100.00
Name: WAG_R_Y, dtype: float64

Тем самым, мы получаем новую выборочную оценку корреляции, не нарушая её смысл (не перемежая значения x и значения w в несоответствующем порядке). Теперь посмотрим на новую выборочную оценку корреляции между показателями:

In [25]:
corr(wag_c_y1,wag_r_y1)

0.9308961717689509

Построив аналогичным образом 10 тыс. бутстрэп-выборок, получаем доверительный бутстрэп-интервал для истинной корреляции:

In [26]:
bootstrap = IIDBootstrap(wag_c_y, wag_r_y, seed=111111)
bootstrap.conf_int(corr, method='basic', reps=10000, size=0.95) #Строим 95%-ый доверительный интервал.

array([[0.90951953],
       [0.96102099]])

5) Подводя итоги, с помощью бутстрэпа мы можем легко оценить статистические параметры выборки, не зная асимптотических формул для этих самых параметров. В данном домашнем задании мы построили 95%-ый доверительный интервал для оценки корреляции между показателями среднегодовой номинальной заработной платы и индексом реальной зарплаты - от 0.9095 до 0.9610.